In [ ]:
with open('input.txt') as file:
    data = file.read()

data = data.split('\n\n')

machines = []
for machine in data:
    rows = machine.split('\n')
    buttons = [[0,0],[0,0]]
    for ii in range(2):
        row = rows[ii].split(': ')[1]
        buttons[0][ii] = int(row.split('+')[1].split(',')[0])
        buttons[1][ii] = int(row.split('+')[2])
    
    row = rows[2].split(': ')[1]
    x = int(row.split('=')[1].split(',')[0])
    y = int(row.split('=')[2])

    machines.append({
        'buttons':buttons,
        'result':(x,y)
    })

In [ ]:
import numpy as np
# Solved using Diophantine equations

def is_whole(arr, eps):
    return all([abs(f - round(f)) < abs(eps) for f in arr])

def gcdExtended(a, b):
    # Extended euclidean algorithm
    if a == 0 : 
        return b,0,1
    gcd,x1,y1 = gcdExtended(b%a, a) 
    x = y1 - (b//a) * x1 
    y = x1 
    return gcd,x,y 

def calc_coefs(x1,x2,x3,y1,y2,y3):
    gcd_x, u_x, v_x = gcdExtended(x1,x2)
    gcd_y, u_y, v_y = gcdExtended(y1,y2)
    
    a,b = -1,-1

    # The result only exists if each of the sums is divisible by the corresponding gcd
    if x3 % gcd_x == 0 and y3 % gcd_y == 0: 
        a0_x = x3/gcd_x * u_x
        b0_x = x3/gcd_x * v_x
        a0_y = y3/gcd_y * u_y
        b0_y = y3/gcd_y * v_y

        # Formed from diaphatines equations
        A = np.array([[x2/gcd_x, - y2/gcd_y],[-x1/gcd_x, y1/gcd_y]])
        b = np.array([a0_y-a0_x, b0_y - b0_x])
        k = np.linalg.solve(A,b)

        if is_whole(k,0.01): # only accept integer solutions
            a = round(a0_x + k[0] * x2/gcd_x)
            b = round(b0_x - k[0] * x1/gcd_x)

    return a,b 

offset = 10000000000000
s,g = 0, 0
for machine in machines:

    # Silver
    x1,x2 = machine['buttons'][0]
    x3 =  machine['result'][0]
    y1,y2 = machine['buttons'][1]
    y3 =  machine['result'][1]
    a,b = calc_coefs(x1,x2,x3,y1,y2,y3)
    if 0 <= a <= 100 and  0 <= b <= 100 :
        s += 3*a + b

    # Gold
    x3 += offset
    y3 += offset
    a,b = calc_coefs(x1,x2,x3,y1,y2,y3)
    if 0 <= a and  0 <= b :
        g += 3*a + b


s,g